<a href="https://colab.research.google.com/github/patty-gz02/SistemaRecomendacion/blob/main/Sistema_Recomendacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejemplo de sistema de Recomendación basado en Filtrado Colaborativo
Un sistema de recomendación es una colección de algoritmos que filtra toda la información disponible sobre un usuario en particular para predecir qué es lo que tendrá valor para él con el fin de que la sugerencia también se convierta en una transacción o para evitar la pérdida de ese cliente.

En este notebook, mostraremos los sistemas de recomendación basados en el filtrado colaborativo e implementaremos una versión simple de uno usando Python y Pandas.

<hr>

<a id="ref2"></a>

# Pre-procesamiento

In [ ]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Leemos cada archivo en un Dataframe:



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Lee la informacion de Libros a un pandas dataframe
libros_df = pd.read_csv('/content/drive/My Drive/Libro.csv',
                      sep=';',
                      encoding='utf-8',
                      na_values=' ')
# lee la informacion de ratings de usuarios e un pandas dataframe
ratings_df = pd.read_csv('/content/drive/My Drive/ratings.csv',
                      sep=';',
                      encoding='utf-8',
                      na_values=' ')

In [ ]:

libros2_df = pd.read_csv('/content/drive/My Drive/Libro.csv',
                      sep=';',
                      encoding='utf-8',
                      na_values=' ')
ratings2_df = pd.read_csv('/content/drive/My Drive/ratings.csv',
                      sep=';',
                      encoding='utf-8',
                      na_values=' ')

In [ ]:
# numero de registros de libros
libros_df.size

12000

In [ ]:
# numero de registros de ratings
ratings_df.size

196605

### Preprocesamiento de la informacion de Libros (items)  

In [ ]:
#muestra los primeros 5 registros de Libros.
libros_df.head(20)

,id,rating,books_count,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count
0,1,1,272,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254
1,2,2,491,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867
2,3,3,226,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009
3,4,4,487,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586
4,5,5,1356,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992
5,6,6,226,John Green,2012.0,The Fault in Our Stars,The Fault in Our Stars,eng,4.26,2346404,2478609,140739
6,7,7,969,J.R.R. Tolkien,1937.0,The Hobbit or There and Back Again,The Hobbit,en-US,4.25,2071616,2196809,37653
7,8,8,360,J.D. Salinger,1951.0,The Catcher in the Rye,The Catcher in the Rye,eng,3.79,2044241,2120637,44920
8,9,9,311,Dan Brown,2000.0,Angels & Demons,"Angels & Demons (Robert Langdon, #1)",en-CA,3.85,2001311,2078754,25112
9,10,10,3455,Jane Austen,1813.0,Pride and Prejudice,Pride and Prejudice,eng,4.24,2035490,2191465,49152


In [ ]:
libros_df.head()

,id,rating,books_count,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count
0,1,1,272,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254
1,2,2,491,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867
2,3,3,226,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009
3,4,4,487,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586
4,5,5,1356,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992


Cada libro tiene un único ID, un título original, autor, un idioma, su año de lanzamiento junto con él  y varios ratings diferentes en el mismo campo.

Vamos extraer la informacion de año de la columna title y y colocarla en una nueva columna year. Para ello usaremos la función extract de Pandas.

In [ ]:
# Eliminar la columna que no utilizaremos
libros_df = libros_df.drop('books_count', 1)
libros_df = libros_df.drop('original_title', 1)
libros_df = libros_df.drop('language_code', 1)
libros_df = libros_df.drop('average_rating', 1)
libros_df = libros_df.drop('ratings_count', 1)
libros_df = libros_df.drop('work_ratings_count', 1)
libros_df = libros_df.drop('work_text_reviews_count', 1)

<ipython-input-9-fdc7047d6f27>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  libros_df = libros_df.drop('books_count', 1)
<ipython-input-9-fdc7047d6f27>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  libros_df = libros_df.drop('original_title', 1)
<ipython-input-9-fdc7047d6f27>:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  libros_df = libros_df.drop('language_code', 1)
<ipython-input-9-fdc7047d6f27>:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  libros_df = libros_df.drop('average_rating', 1)
<ipython-input-9-fdc7047d6f27>:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' 

In [ ]:
libros_df.head()

,id,rating,authors,original_publication_year,title
0,1,1,Suzanne Collins,2008.0,"The Hunger Games (The Hunger Games, #1)"
1,2,2,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Sorcerer's Stone (Harry P...
2,3,3,Stephenie Meyer,2005.0,"Twilight (Twilight, #1)"
3,4,4,Harper Lee,1960.0,To Kill a Mockingbird
4,5,5,F. Scott Fitzgerald,1925.0,The Great Gatsby


Preprocesamiento de la informacion de ratings (matriz users-items)

In [ ]:
ratings_df.head()

,book_id,user_id,rating
0,1.0,314.0,5.0
1,2.0,439.0,3.0
2,3.0,588.0,5.0
3,4.0,1169.0,4.0
4,5.0,1185.0,4.0


Cada fila en el dataframe en ratings tiene una identificación del libro al que esta asociado, una identificaicon del usuario que realizo la encuesta. Como no se usa esta columna, la borramos para ahorrar memoria.

In [ ]:
ratings_df = ratings_df.drop('user_id', 1)

<ipython-input-47-350ebb2c2b73>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  ratings_df = ratings_df.drop('user_id', 1)


In [ ]:
ratings_df.head()

,book_id,rating
0,1.0,5.0
1,2.0,3.0
2,3.0,5.0
3,4.0,4.0
4,5.0,4.0


<hr>

<a id="ref3"></a>

# Sistema de Recomendacion con Filtrado Colaborativo basado en usuario

El **Filtrado colaborativo** sirve para hacer predicciones automáticas sobre los intereses de un usuario mediante la recopilación de preferencias o gustos del mismo usuario o de otros usuarios con intereses similares.
Tambien sirve para hacer recomendaciones sobre items de interes.

Cuando esta técnica utiliza las relaciones que suceden entre usuarios con preferencias similares para recomendar items de interes al usuario de consulta, se denomina **filtros basados en usuarios** y cuando la técnica usa las relaciones entre usuarios y productos a recomendar se les conoce como **filtros basados en items**.

El proceso para crear un sistema de recomendación basado en el usuario para recomendacion de libros es el siguiente:

- Obtener la informacion del usuario de consulta sobre items que ha evaluado (libros mas leidos).
- Según su calificación de los items (libros), encontrar los X  usuarios vecinos mas similares
- Por cada vecino obtener informacion de los items vistos por el usuario.
- Calcular una puntuación de similitud usando alguna medida de distancia o similaridad.
- Recomendar los ítems (libros) no vistos con la puntuación más alta.

In [ ]:
userInput = [
            {'authors':'Gabriel García Márquez','original_publication_year':1967.0,'title':'Cien años de soledad'},
            {'authors':'J. R. R. Tolkien','original_publication_year':2001.0,'title':'El señor de los anillos (Trilogía)'},
            {'authors':'George Orwell','original_publication_year':1984,'title':'1984'},
            {'authors':'Aldous Huxley','original_publication_year':1932.0,'title':"Un mundo feliz"},
            {'authors':'Fiódor Dostoyevski','original_publication_year':1866.0,'title':"Crimen y castigo"}
]
inputBooks = pd.DataFrame(userInput)
inputBooks



,authors,original_publication_year,title
0,Gabriel García Márquez,1967.0,Cien años de soledad
1,J. R. R. Tolkien,2001.0,El señor de los anillos (Trilogía)
2,George Orwell,1984.0,1984
3,Aldous Huxley,1932.0,Un mundo feliz
4,Fiódor Dostoyevski,1866.0,Crimen y castigo


#### Colocamos el movieId a cada item de la consulta del usuario

Ahora se extrae y adicionan los ID de movies dataframe
También eliminamos columnas innecesarias para ahorrar memoria.

In [ ]:
# Buscamos en la columna title del dataframe  libros_df
inputId = libros_df[libros_df['title'].isin(inputBooks['title'].tolist())]
# Junta el resultado de la busqueda con el dataframe de la consulta de usuario (inputBooks). Esta implicito que el merge es por la columna title.
inputBooks = pd.merge(inputId, inputBooks)
inputBooks

,id,rating,authors,original_publication_year,title


#### Buscamos usuarios similares que han leidos los mismos libros

Con los ID de los libros se obtiene el subconjunto de usuarios que han visto y calificado los mismos.

In [ ]:
# Busca registros en dataframe ratings_df cuyo book_id  estan en los libros de la consulta del usuario de entrada
userSubset = ratings_df[ratings_df['book_id'].isin(ratings2_df['user_id'].tolist())]
userSubset.head(5)

,book_id,rating
172,173.0,4.0
313,314.0,4.0
438,439.0,2.0
587,588.0,5.0
724,725.0,4.0


In [ ]:
userSubset.size

129082

Ahora agrupamos el dataframe resultante por ID de usuario.

In [ ]:
# Groupby crea sub dataframes, cada uno teniendo el mismo valor en la columna especificada como parametro
userSubsetGroup = userSubset.groupby(['book_id'])

Veamos algunos de los vecinos, p. ej. el que tiene userID = 6
<br><br>
Ordenemos estos grupos para que los usuarios que comparten la mayor cantidad de películas en común con el usuario de consulta tengan una prioridad más alta. Esto proporciona una recomendación más completa, ya que no revisaremos a todos los usuarios.

In [ ]:
# Ordena a los usuarios de acuerdo a la cantidad de items evaluados coincidentes con el usuario de consulta (a mayor cantidad  mayor prioridad)
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

In [ ]:
userSubsetGroup[0:5]

[(173.0,      book_id  rating
  172    173.0     4.0), (314.0,      book_id  rating
  313    314.0     4.0), (439.0,      book_id  rating
  438    439.0     2.0), (588.0,      book_id  rating
  587    588.0     5.0), (725.0,      book_id  rating
  724    725.0     4.0)]

#### Usuarios similares al usuario de consulta

A continuación, vamos a comparar los usuarios (¡no todos!) con nuestro usuario de consulta y encontrar los que sean más similares segun sus ratings.

Usaremos el **Coeficiente de correlación de Pearson**.
Este coeficiente mide la intensidad de asociación entre dos variables. La fórmula para encontrar la correlación entre las variables X y Y con N valores N es la siguiente:

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/bd1ccc2979b0fd1c1aec96e386f686ae874f9ec0 "Pearson Correlation")

El coeficiente r varia entre [-1 y 1]. Cuando r=1 la correlación es directa entre las dos variables (significa una correlación positiva perfecta) y si r= -1 la correlación es negativa perfecta.

En nuestro caso, un 1 significa que dos usuarios tienen gustos correlacionados, mientras que un -1 significa lo contrario.


Seleccionaremos un subconjunto de usuarios a iterar.

In [ ]:
# seleccionamos los 10 usuarios con mas cantidad de items evaluados coincidentes con el usuario de consulta
userSubsetGroup = userSubsetGroup[0:10]

NameError: ignored

Calculamos la correlación de Pearson entre el usuario de consulta y el grupo de subconjuntos. Esto se guarda en un diccionario, donde la clave es el ID del usuario y el valor y es el coeficiente r.


In [ ]:
#Almacena la correlacion de Pearson en un diccionario, la llave es el userId y el value es el coeficiente

pearsonCorrelationDict = {}

# Por cada grupo (usuario vecino) en nuestro  userSubsetGroup
for name, group in userSubsetGroup:
    # Ordena los libros del grupo y los ratings del usuario de consulta
    group = group.sort_values(by='book_id')
    libros_df = libros2_df.sort_values(by='rating')
    # Obtiene el valor n de la formula de Pearson
    nRatings = len(group)
    # Obtiene los items (libros) en comun entre el grupo y los ratings del usuario de consulta
    temp_df = ratings2_df[ratings2_df['rating'].isin(group['book_id'].tolist())]
    # Extrae los rating de los libros en comun en una lista
    tempRatingList = temp_df['rating'].tolist()
    # Extrae los ratings de los libros del grupo (usuario vecino) en una lista
    tempGroupList = group['rating'].tolist()
    # Ahora calcula la correlacion de pearson entre los dos usuarios, llamemosle, x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)

    #Si el denominador es diferente de 0 entonces divide, sino la correlacion es 0.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0


In [ ]:
pearsonCorrelationDict.items()

In [ ]:
# Convierte el diccionario de correlaciones en un DataFrame
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head(5)

#### Ordena los usuarios segun su similaridad al usuario de consulta

Mostramos los usuarios en orden decresciente segun su similaridad (coeficiente de pearson) al usuario de consulta .

In [ ]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)
topUsers.head(5)

NameError: ignored

### Recomendación de películas para el usuario de consulta

Iniciamos tomando el promedio ponderado de las calificaciones de las películas usando la Correlación de Pearson como peso.
Antes es necesario obtener las películas vistas por los usuarios mas similares de ** pearsonDF ** en el dataframe de calificaciones (ratings) y luego esta correlación se guarda en una nueva columna llamada "similarityIndex".

In [ ]:
# para los usuarios mas similafres, obtiene sus registros de ratings_df de todos los libros que ha visto y anexa columna "similarityIndex" del usuario
topUsersRating = ratings_df.merge(ratings_df, left_on='book_id', right_on='book_id', how='inner')
topUsers.head()

Ahora se multiplica la calificación de la película por su peso (el índice de similitud), luego se suma las nuevas calificaciones y se divide por la suma de los pesos.

Podemos hacer esto simplemente multiplicando dos columnas, luego agrupando el dataframe por movieId y despues dividir en dos columnas:

Esto nos muestra todos los usuarios con gustos similares sobre las películas del usuario de consulta:


In [ ]:
# Multiplica la similaridad por el rating del usuario y el resultado lo pon e en una columna weightedRating
topUsers['weightedRating'] = topUsers['similarityIndex']*topUsers['userId']
topUsers.head()

,similarityIndex,userId,weightedRating
0,0,173.0,0.0
1,0,314.0,0.0
2,0,439.0,0.0
3,0,588.0,0.0
4,0,725.0,0.0


In [ ]:
#Suma los weightedRating y similarityIndex para cada item (movieId). La suma es a lo largo de los usuarios
tempTopUsersRating = topUsers.groupby('userId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
userId,,
173.0,0,0.0
314.0,0,0.0
439.0,0,0.0
588.0,0,0.0
725.0,0,0.0


In [ ]:
# Crea un dataframe vacio
recommendation_df = pd.DataFrame()
# Ahora obtenemos el score final de recomendacion de cada item como el ratio sum_weightedRating/sum_similarityIndex
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['bookId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,bookId
userId,,
173.0,NaN,173.0
314.0,NaN,314.0
439.0,NaN,439.0
588.0,NaN,588.0
725.0,NaN,725.0


#### Recomendacion final
Ordena los items (películas) de acuerdo a su score final de recomendacionmn ('weighted average recommendation score').

In [ ]:
# ordena items en orden descendente 'weighted average recommendation score'
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,bookId
userId,,
173.0,NaN,173.0
314.0,NaN,314.0
439.0,NaN,439.0
588.0,NaN,588.0
725.0,NaN,725.0
951.0,NaN,951.0


In [ ]:
# muestra los nombres de las peliculas recomendadas
libros_df.loc[libros_df['id'].isin(recommendation_df.head(10)['bookId'].tolist())]

,id,rating,books_count,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count
172,173,173,283,Anthony Burgess,1962.0,A Clockwork Orange,A Clockwork Orange,eng,3.98,431195,454391,9958
313,314,314,134,"Cornelia Funke, Anthea Bell",2003.0,Tintenherz,"Inkheart (Inkworld, #1)",en-US,3.86,285011,301414,10384
438,439,439,246,"Stephen Hawking, Cao Chi, Ph?m V?n Thi?u",1987.0,A Brief History of Time: From the Big Bang to ...,A Brief History of Time,eng,4.12,165628,181410,5146
587,588,588,101,Robert Ludlum,1986.0,The Bourne Supremacy,"The Bourne Supremacy (Jason Bourne, #2)",en-US,4.09,142168,145806,1010
724,725,725,151,"Stephen King, Ned Dameron",1991.0,The Waste Lands,"The Waste Lands (The Dark Tower, #3)",eng,4.24,117483,135257,3844
950,951,951,63,Chuck Palahniuk,1999.0,Survivor,Survivor,en-US,3.93,49551,88231,3078
